### Juster-Maker:
Tool allowing to run events in cycle

In [1]:
from pytezos import pytezos
import time
from random import choice
from pprint import pprint
from random import randint

# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

### - read is there any nice light schedulers in python?

In [2]:
# CreateEventTask

In [3]:
event_params = {
    'freq': 1,
    'bets_duration': 300,
    'measurement_duration': 300,
    'pair': 'XTZ-USD',
    'target_dynamics': 1_000_000,
    'liquidity_percent': 10_000
}

In [4]:
def create_event():
    pass

def repeat_untill_succeed(func, allow_exceptions=None):
    pass

def make_event_creator(client):

    def event_creator():
        return repeat_untill_succeed(
            lambda client: create_event(client))

    return event_creator

def schedule_creator(period, lambda client: create_event(client))

SyntaxError: invalid syntax (<ipython-input-4-0d44fac25188>, line 15)

In [5]:
import sched

In [6]:
class EventCreator:
    def __init__(self, contract, period, event_params):
        """ Initializes new event creator
            - contract: is pytezos object with Juster contract loaded and
                some key provided
            - period: period in seconds  """

        self.contract = contract
        self.event_params = event_params
        self.freq = freq

    def create(self):
        fees = (
            keys[key].storage()['config']['expirationFee']
            + keys[key].storage()['config']['measureStartFee'])

    def create_untill_succeed(self):
        pass


def create_event(client, ):
    event_params = {
        'currencyPair': choice(CURRENCY_PAIRS),
        'targetDynamics': choice(TARGET_DYNAMICS),
        'betsCloseTime': choice(BETS_CLOSE_TIME),
        'measurePeriod': choice(MEASURE_PERIOD),
        'liquidityPercent': choice(LIQUIDITY_PERCENT),
    }

    key = pick_random_free_key(keys, last_used)
    print(f'creating event with {key}:')
    pprint(event_params)

    fees = (
        keys[key].storage()['config']['expirationFee']
        + keys[key].storage()['config']['measureStartFee'])

    keys[key].newEvent(event_params).with_amount(fees).as_transaction().autofill().sign().inject(_async=False)
    return event_params


In [25]:
import asyncio
from abc import abstractmethod
from asyncio import Queue


async def repeat_until_succeed(
        func, allowed_exceptions=None, max_attempts=10, wait_after_fail=10):
    """ Runs func for multiple times if it was failed with any of the allowed
        exceptions """

    allowed_exceptions = allowed_exceptions or []

    for attempt in range(max_attempts):
        try:
            return await func()

        except Exception as e:
            if type(e) in allowed_exceptions:
                print(f'Ignoring error {type(e)}, {str(e)}')
                await asyncio.sleep(wait_after_fail)
            else:
                raise e


class LoopExecutor:
    """ Executes self.execute each period seconds """

    def __init__(self, period):
        self.period = period
        self.loop = asyncio.get_event_loop()


    def run(self):
        self.task = self.loop.create_task(self.loop_task())


    @abstractmethod
    async def execute(self):
        raise NotImplemented('method execute should be implemented')


    async def loop_task(self):
        while True:
            await self.execute()
            await asyncio.sleep(self.period)


    def stop(self):
        self.task.cancel()


class EventCreationEmitter(LoopExecutor):

    def __init__(self, period, operations_queue, event_params):
        super().__init__(period)

        self.operations_queue = operations_queue
        self.event_params = event_params


    async def create_event(self):
        await self.operations_queue.put(self.event_params)
        # TODO: not implemented
        # TODO: calculate event params
        # event_params = None

        print(f'called create_event with params: {self.event_params}')


    async def execute(self):
        return await repeat_until_succeed(self.create_event)


class BulkSender(LoopExecutor):
    """ Listens to the queue and packs all new operations to the bulk,
        signs and sends the transaction
    """

    def __init__(self, period, client, operations_queue):
        super().__init__(period)

        self.client = client
        self.operations_queue = operations_queue


    async def sign(self, max_operations=3):
        # TODO: check if ready to sign
        # ? await self.is_ready_to_sign()

        operations = []
        while (not self.operations_queue.empty()
               and (len(operations) < max_operations)):
            operations.append(await operations_queue.get())

        try:
            print('SIGNED: ', operations)
        except Exception as e:
            print(f'catched {type(e)} in sign: {str(e)}')
            # TODO: here I need to classify error and if it was RPC error: return
            # operations back to the queue, if not: raise this e
            import pdb; pdb.set_trace()
            raise e


    async def is_ready_to_sign(self, sleep_time=90):
        """ Waits while client would be possible to sign transaction
            - simple solution is just timer, better would be to reduce 
                this sleep_time value and somehow ask RPC if key is freed
        """

        return await asyncio.sleep(sleep_time)


    async def execute(self):
        return await repeat_until_succeed(self.sign)


# make queue with transactions
# make bulk manager that takes transactions and send them into blocks

In [31]:
# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 10

client = None
operations_queue = Queue(MAX_QUEUE_SIZE)

emitters = [
    EventCreationEmitter(period=1, operations_queue=operations_queue, event_params='test'),
    EventCreationEmitter(period=3, operations_queue=operations_queue, event_params='RARE'),
    BulkSender(period=4, client=client, operations_queue=operations_queue)
]

In [32]:
for emitter in emitters:
    emitter.run()

called create_event with params: test
called create_event with params: RARE
SIGNED:  ['test', 'RARE']
called create_event with params: test
called create_event with params: test
called create_event with params: RARE
called create_event with params: test
SIGNED:  ['test', 'test', 'RARE']
called create_event with params: test
called create_event with params: test
called create_event with params: RARE
called create_event with params: test
called create_event with params: test
SIGNED:  ['test', 'test', 'test']
called create_event with params: test
called create_event with params: RARE
called create_event with params: test
called create_event with params: test
called create_event with params: test
SIGNED:  ['RARE', 'test', 'test']
called create_event with params: RARE
called create_event with params: test
called create_event with params: test
called create_event with params: test
called create_event with params: RARE
SIGNED:  ['test', 'RARE', 'test']
called create_event with params: test
ca

In [33]:
for emitter in emitters:
    emitter.stop()

In [ ]:
task = loop.create_task(emitter.run())

In [3]:


CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}

In [4]:
pytezos


Properties
.key		tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm
.shell		['https://api.tez.ie/rpc/florencenet']
.block_id	head

Helpers
.account()
.activate_account()
.activate_protocol()
.bake_block()
.balance()
.ballot()
.bulk()
.check_message()
.contract()
.delegation()
.double_baking_evidence()
.double_endorsement_evidence()
.endorsement()
.endorsement_with_slot()
.failing_noop()
.now()
.operation()
.operation_group()
.origination()
.proposals()
.reveal()
.seed_nonce_revelation()
.sign_message()
.transaction()
.using()
.wait()

In [ ]:
# using pytezos mock:

# Test event is created:

# Test 